# USING TORCHEEG TO COMPLETE A DEEP LEARNING WORKFLOW

In [1]:
# import torch
# import torch.nn as nn

# x = torch.randn(4, 128, 10, 10, 10)  # (B, C, D, H, W)

# pool = nn.MaxPool3d(kernel_size=(1, 2, 2), stride=(1, 2, 2))
# y = pool(x)

# print(y.shape)  # torch.Size([4, 128, 10, 5, 5])


In this tutorial, we demonstrate a complete deep learning workflow using TorchEEG. We will cover the following aspects:

Utilizing Datasets and Transformers in TorchEEG

Data Partitioning Strategies in TorchEEG

Leveraging Models and Trainers in TorchEEG

In [2]:
!py --version

Python 3.10.11


In [3]:
import torch, torch_scatter; print(torch.__version__, torch_scatter.__version__)

2.5.1+cu121 2.1.2+pt25cu121


In [4]:
!pip install torcheeg
!pip install ipywidgets


  Using cached ipywidgets-8.1.8-py3-none-any.whl.metadata (2.4 kB)
  Using cached widgetsnbextension-4.0.15-py3-none-any.whl.metadata (1.6 kB)
  Using cached jupyterlab_widgets-3.0.16-py3-none-any.whl.metadata (20 kB)
Using cached ipywidgets-8.1.8-py3-none-any.whl (139 kB)
Using cached jupyterlab_widgets-3.0.16-py3-none-any.whl (914 kB)
Using cached widgetsnbextension-4.0.15-py3-none-any.whl (2.2 MB)

   -------------------------- ------------- 2/3 [ipywidgets]
   -------------------------- ------------- 2/3 [ipywidgets]
   -------------------------- ------------- 2/3 [ipywidgets]
   ---------------------------------------- 3/3 [ipywidgets]



In [5]:
!pip install torch-scatter -f https://data.pyg.org/whl/torch-2.6.0+cu124.html

Looking in links: https://data.pyg.org/whl/torch-2.6.0+cu124.html


## Step 1: Initialize the Dataset

We use the DEAP dataset supported by TorchEEG. Each EEG sample is set to be 1 second long, encompassing 128 data points. The baseline signal is 3 seconds long, which we divide into three sections and then average to obtain the trial’s baseline signal.

During offline preprocessing, we divide each electrode’s EEG signal into 4 sub-bands, calculate the differential entropy for each sub-band as a feature, perform debaselining, and map onto a grid. Finally, the preprocessed EEG signals are saved locally. For online processing, we convert all EEG signals into Tensors, making them suitable for neural network input.

In [6]:
from torcheeg.datasets import DEAPDataset
from torcheeg import transforms

from torcheeg.datasets.constants.emotion_recognition.deap import \
    DEAP_CHANNEL_LOCATION_DICT

# dataset = DEAPDataset(
#     io_path = r"/kaggle/working/.torcheeg/datasets_1764615398173_N2O6z", # grid data 4x9x9
#     root_path=r"/kaggle/input/deap-dataset/deap-dataset/data_preprocessed_python",
#     offline_transform=transforms.Compose([
#         transforms.BaselineRemoval(),
#         transforms.BandDifferentialEntropy(apply_to_baseline=True),
#         transforms.MeanStdNormalize(apply_to_baseline=True),
#         transforms.ToGrid(DEAP_CHANNEL_LOCATION_DICT, apply_to_baseline=True)
#     ]),
#     online_transform=transforms.Compose(
#         [transforms.ToTensor()]),
#     label_transform=transforms.Compose([
#         transforms.Select('valence'),
#         transforms.Binary(5.0),
#     ]))



dataset = DEAPDataset(
    io_path = r".torcheeg/datasets_1769886804104_NjcHF", # grid data 4x9x9
    root_path=r"C:\Users\pc\Root\programming\eeg\deap_signals\deap-dataset\data_preprocessed_python",
     offline_transform=transforms.Compose([
        transforms.BandDifferentialEntropy(apply_to_baseline=True),
        transforms.ToGrid(DEAP_CHANNEL_LOCATION_DICT, apply_to_baseline=True)
    ]),
    online_transform=transforms.Compose(
        [transforms.BaselineRemoval(),
         transforms.ToTensor()]),
    label_transform=transforms.Compose([
        transforms.Select('valence'),
        transforms.Binary(5.0),
    ]))



[2026-02-01 20:43:59] INFO (torcheeg/MainThread) 🔍 | Detected cached processing results, reading cache from .torcheeg/datasets_1769886804104_NjcHF.


In [7]:
eeg_tensor, label = dataset[0]
print(eeg_tensor.shape)

torch.Size([4, 9, 9])


In [8]:
from collections import Counter

counts = Counter(label for _, label in dataset)
zeros = counts[0]
ones = counts[1]

print(f"zeros: {zeros}")
print(f"ones: {ones}")

zeros: 59700
ones: 17100


In [9]:
from torcheeg.model_selection import train_test_split_groupby_trial

train_dataset, val_and_test_dataset = train_test_split_groupby_trial(
    dataset = dataset, test_size = 0.2
)

val_dataset, test_dataset = train_test_split_groupby_trial(
    dataset = val_and_test_dataset, test_size = 0.5
)

[2026-02-01 20:44:04] INFO (torcheeg/MainThread) 📊 | Create the split of train and test set.
[2026-02-01 20:44:04] INFO (torcheeg/MainThread) 😊 | Please set split_path to .torcheeg\model_selection_1769971444157_smVDN for the next run, if you want to use the same setting for the experiment.
[2026-02-01 20:44:07] INFO (torcheeg/MainThread) 📊 | Create the split of train and test set.
[2026-02-01 20:44:07] INFO (torcheeg/MainThread) 😊 | Please set split_path to .torcheeg\model_selection_1769971447384_1ggwr for the next run, if you want to use the same setting for the experiment.


In [10]:
import torch
from torch import nn

def conv(channels_in, channels_out): 
    return nn.Conv2d(channels_in, channels_out, kernel_size = 3, stride = 1, padding = 'same', bias = False)

def conv_down(channels_in, channels_out): 
    return nn.Conv2d(channels_in, channels_out, kernel_size = 3, stride = 3, bias = False)

def pool(channels_in, channels_out): 
    return nn.Conv2d(channels_in, channels_out, kernel_size = 3, stride = 3, bias = False)

def conv1x1(channels_in, channels_out): 
    return nn.Conv2d(channels_in, channels_out, kernel_size = 1, stride = 1, padding = 'same')

def bn(channels_in): 
    return nn.BatchNorm2d(channels_in)

def relu():
    return nn.ReLU()

def dropout(rate):
    return nn.Dropout(rate)




class Layer(nn.Sequential):
    def __init__(self, channels):
        super(Layer, self).__init__()
        self.add_module('norm', bn(channels))
        self.add_module('relu', relu())
        self.add_module('conv', conv(channels, channels))


class ResBlock(nn.Module):
    def __init__(self, channels, number_of_layers):
        super(ResBlock, self).__init__()
        for i in range(number_of_layers):
            layer = Layer(channels = channels)
            self.add_module('layer%d' % (i + 1), layer)
    
    def forward(self, input_data):
        features = input_data
        for _, layer in self.named_children():
            features = layer(features) + features            
        return features


class Transition_Layer(nn.Sequential):
    def __init__(self, channels_in, channels_out):
        super(Transition_Layer, self).__init__()
        self.add_module('norm', bn(channels_in))
        self.add_module('relu', nn.ReLU())
        self.add_module('conv1x1', conv1x1(channels_in, channels_out))
        self.add_module('pool', pool(channels_out, channels_out))
        # self.add_module('dropout', dropout(0.05))


class stage(nn.Sequential):
    def __init__(self, channels_in, channels_out, number_of_layers):
        super(stage, self).__init__()
        self.add_module('res_block', ResBlock(channels_in, number_of_layers))
        self.add_module('transition_layer', Transition_Layer(channels_in, channels_out))


class final_stage(nn.Sequential):
    def __init__(self, channels_in, channels_out, number_of_layers):
        super(final_stage, self).__init__()
        self.add_module('res_block', ResBlock(channels_in, number_of_layers))
        self.add_module('conv1x1', conv1x1(channels_in, channels_out))
        self.add_module('pool', conv_down(channels_in, channels_out))
        self.add_module('flatten', nn.Flatten())

class first_linear(nn.Sequential):
    def __init__(self, length_in, length_out):
        super(first_linear, self).__init__()
        self.add_module('linear', nn.Linear(length_in, length_out))
        self.add_module('norm', nn.BatchNorm1d(length_out))
        self.add_module('relu', relu())
        # self.add_module('dropout', dropout(0.25))

class ResNet(nn.Sequential):
    def __init__(self):
        super(ResNet, self).__init__()
        # self.add_module('padding1', nn.ZeroPad2d(1)) # 9 -> 10
        # self.add_module('stage1', stage(4, 32, 3)) # 9 -> 3
        self.add_module('resBlock1', ResBlock(4, 3)) # 5 -> 5
        self.add_module('conv1x1_1', conv1x1(4, 32)) # 5 -> 5
        self.add_module('resBlock2', ResBlock(32, 2)) # 5 -> 5
        self.add_module('conv1x1_2', conv1x1(32, 64)) # 5 -> 5
        self.add_module('resBlock3', ResBlock(64, 2)) # 5 -> 5
        # self.add_module('resBlock1', ResBlock(32, 3)) # 5 -> 5
        # self.add_module('resBlock2', ResBlock(32, 3)) # 5 -> 5
        # self.add_module('padding2', nn.ZeroPad2d(1) ) # 5 -> 6
        # self.add_module('stage2', stage(32, 32, 3)) # 3 -> 1
        self.add_module('flatten', nn.Flatten()) # 1x1x32 = 288 
        self.add_module('linear1', first_linear(9*9*64, 2048))
        self.add_module('final_linear', nn.Linear(2048, 2))

In [11]:
!pip install torchinfo

In [12]:
from torchinfo import summary

model = ResNet()
summary(model, input_size=(64, 4, 9, 9))

Layer (type:depth-idx)                   Output Shape              Param #
ResNet                                   [64, 2]                   --
├─ResBlock: 1-1                          [64, 4, 9, 9]             --
│    └─Layer: 2-1                        [64, 4, 9, 9]             --
│    │    └─BatchNorm2d: 3-1             [64, 4, 9, 9]             8
│    │    └─ReLU: 3-2                    [64, 4, 9, 9]             --
│    │    └─Conv2d: 3-3                  [64, 4, 9, 9]             144
│    └─Layer: 2-2                        [64, 4, 9, 9]             --
│    │    └─BatchNorm2d: 3-4             [64, 4, 9, 9]             8
│    │    └─ReLU: 3-5                    [64, 4, 9, 9]             --
│    │    └─Conv2d: 3-6                  [64, 4, 9, 9]             144
│    └─Layer: 2-3                        [64, 4, 9, 9]             --
│    │    └─BatchNorm2d: 3-7             [64, 4, 9, 9]             8
│    │    └─ReLU: 3-8                    [64, 4, 9, 9]             --
│    │    └─Conv

In [13]:
from torch.utils.data import DataLoader
from torcheeg.models import CCNN
from torcheeg.models import ViT
from torcheeg.trainers import ClassifierTrainer, WCELossTrainer
import pytorch_lightning as pl
# import mlflow

# mlflow.pytorch.autolog()

train_loader = DataLoader(train_dataset, batch_size = 64, shuffle = True)
val_loader = DataLoader(val_dataset, batch_size = 64, shuffle = False)
test_loader = DataLoader(test_dataset, batch_size = 64, shuffle = False)

# model = ViT(chunk_size=4,
#             grid_size=(9, 9),
#             t_patch_size=1,
#             num_classes=2)

# model = CCNN(num_classes = 2, in_channels = 4, grid_size = (9, 9))

model = ResNet()

trainer = ClassifierTrainer(
    model=model,
    num_classes=2,
    lr=1e-4,
    weight_decay=1e-4,
    accelerator="gpu",
    metrics = ['precision', 'recall', 'f1score', 'accuracy', 'kappa']
)

# trainer = WCELossTrainer(
#     model = model,
#     num_classes = 2,
#     class_frequency = [zeros, ones],
#     lr = 1e-4,
#     weight_decay = 1e-4,
#     accelerator = "gpu",
#     metrics = ['precision', 'recall', 'f1score', 'accuracy', 'kappa']
# )

trainer.fit(
    train_loader,
    val_loader,
    max_epochs = 50,
    # default_root_dir = f'./model',
    # callbacks = [pl.callbacks.ModelCheckpoint(save_last=True)],
    enable_progress_bar = True,
    enable_model_summary = True,
)

trainer.test(
    test_loader,
    enable_progress_bar = True,
    enable_model_summary = True    
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
c:\Users\pc\Root\programming\eeg\deap_signals\eeg_env\lib\site-packages\pytorch_lightning\trainer\connectors\logger_connector\logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
💡 Tip: For seamless cloud logging and experiment tracking, try installing [litlogger](https://pypi.org/project/litlogger/) to enable LitLogger, which logs metrics and artifacts automatically to the Lightning Experiments platform.
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCh

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

c:\Users\pc\Root\programming\eeg\deap_signals\eeg_env\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:434: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
[2026-02-01 20:45:35] INFO (torcheeg/MainThread) 
[Val] val_loss: 0.665 val_precision: 0.909 val_recall: 0.521 val_f1score: 0.490 val_accuracy: 0.820 val_kappa: 0.066 
c:\Users\pc\Root\programming\eeg\deap_signals\eeg_env\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:434: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

[2026-02-01 20:56:35] INFO (torcheeg/MainThread) 
[Val] val_loss: 0.107 val_precision: 0.961 val_recall: 0.942 val_f1score: 0.951 val_accuracy: 0.967 val_kappa: 0.902 
[2026-02-01 20:56:35] INFO (torcheeg/MainThread) 
[Train] train_loss: 0.010 train_precision: 0.996 train_recall: 0.995 train_f1score: 0.996 train_accuracy: 0.997 train_kappa: 0.991 
`Trainer.fit` stopped: `max_epochs=50` reached.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
💡 Tip: For seamless cloud logging and experiment tracking, try installing [litlogger](https://pypi.org/project/litlogger/) to enable LitLogger, which logs metrics and artifacts automatically to the Lightning Experiments platform.
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Users\pc\Root\programming\eeg\deap_signals

Testing: |          | 0/? [00:00<?, ?it/s]

[2026-02-01 20:56:36] INFO (torcheeg/MainThread) 
[Test] test_loss: 0.117 test_precision: 0.957 test_recall: 0.939 test_f1score: 0.948 test_accuracy: 0.964 test_kappa: 0.895 


────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_accuracy         0.9644531011581421
      test_f1score          0.9476737380027771
       test_kappa           0.8953681588172913
        test_loss           0.11747756600379944
     test_precision         0.9566860198974609
       test_recall           0.93937087059021
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.11747756600379944,
  'test_precision': 0.9566860198974609,
  'test_recall': 0.93937087059021,
  'test_f1score': 0.9476737380027771,
  'test_accuracy': 0.9644531011581421,
  'test_kappa': 0.8953681588172913}]

In [ ]:
# Hyperparameters
learning_rate = 1e-4
batch_size = 64
num_epochs = 200
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
baseline_num = 128
norm_axis = -1

In [ ]:
# model = ResNet()
model = model.to(device)

In [ ]:
criterion = torch.nn.CrossEntropyLoss(weight=torch.Tensor([zeros, ones]).to(device))

optimizer = torch.optim.Adam(
    model.parameters(),
    lr = learning_rate,
    weight_decay = 1e-4
)

# scheduler = torch.optim.lr_scheduler.OneCycleLR(
#     optimizer,
#     max_lr = 0.1,
#     steps_per_epoch = len(train_loader),
#     epochs = num_epochs
# )

In [ ]:
import torch
from torchmetrics import Accuracy, Precision, Recall, F1Score, CohenKappa

num_classes = 2  
metric_kwargs = dict(task="binary", num_classes=num_classes)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


def make_metrics():
    return {
        "accuracy": Accuracy(**metric_kwargs).to(device),
        "precision": Precision(**metric_kwargs).to(device),
        "recall":    Recall(**metric_kwargs).to(device),
        "f1":        F1Score(**metric_kwargs).to(device),
        "kappa":     CohenKappa(task="binary", num_classes=num_classes).to(device)
    }

train_metrics = make_metrics()
val_metrics = make_metrics()
test_metrics = make_metrics()

# Pretty printing helper
def format_metrics(loss, stats):
    return (
        f"Loss: {loss:.4f} | "
        f"Accuracy: {stats['accuracy']:.4f} | "
        f"Precision: {stats['precision']:.4f} | "
        f"Recall: {stats['recall']:.4f} | "
        f"F1-Score: {stats['f1']:.4f} | "
        f"Kappa: {stats['kappa']:.4f}"
    )

In [ ]:
from tqdm import tqdm

for epoch in range(num_epochs):
    model.train()
    train_loss_total = 0

    for m in train_metrics.values():
        m.reset()

    for data, targets in tqdm(train_loader):
        data, targets = data.to(device), targets.to(device)
    
        scores = model(data)
        loss = criterion(scores, targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss_total += loss.item()
        preds = torch.softmax(scores, dim=1)[:, 1]
        for m in train_metrics.values():
            m.update(preds, targets)

    train_loss_avg = train_loss_total / len(train_loader)
    train_stats = {name: m.compute().item() for name, m in train_metrics.items()}

    print(f"\nEpoch {epoch+1} — Train")
    print(" ", format_metrics(train_loss_avg, train_stats))

    model.eval()
    val_loss_total = 0

    for m in val_metrics.values():
        m.reset()

    with torch.no_grad():
        for data, targets in tqdm(val_loader):
            data, targets = data.to(device), targets.to(device)
    
            scores = model(data)
            val_loss = criterion(scores, targets)
            val_loss_total += val_loss.item()

            preds = torch.softmax(scores, dim=1)[:, 1]
            for m in val_metrics.values():
                m.update(preds, targets)

    val_loss_avg = val_loss_total / len(val_loader)
    val_stats = {name: m.compute().item() for name, m in val_metrics.items()}

    print(f"Epoch {epoch+1} — VAL")
    print(" ", format_metrics(val_loss_avg, val_stats))

    # scheduler.step(val_loss_avg)

100%|██████████| 960/960 [00:20<00:00, 47.69it/s]



Epoch 1 — Train
  Loss: 0.2710 | Accuracy: 0.7773 | Precision: 0.0000 | Recall: 0.0000 | F1-Score: 0.0000 | Kappa: 0.0000


100%|██████████| 120/120 [00:01<00:00, 62.11it/s]


Epoch 1 — VAL
  Loss: 0.3113 | Accuracy: 0.7773 | Precision: 0.0000 | Recall: 0.0000 | F1-Score: 0.0000 | Kappa: 0.0000


100%|██████████| 960/960 [00:19<00:00, 48.06it/s]


...

100%|██████████| 960/960 [00:20<00:00, 47.75it/s]



Epoch 4 — Train
  Loss: 0.2670 | Accuracy: 0.7773 | Precision: 0.0000 | Recall: 0.0000 | F1-Score: 0.0000 | Kappa: 0.0000


100%|██████████| 120/120 [00:01<00:00, 61.85it/s]


Epoch 4 — VAL
  Loss: 0.3050 | Accuracy: 0.7773 | Precision: 0.0000 | Recall: 0.0000 | F1-Score: 0.0000 | Kappa: 0.0000


 28%|██▊       | 265/960 [00:05<00:14, 47.88it/s]

In [ ]:
model.eval()
test_loss_total = 0

for m in test_metrics.values():
    m.reset()

with torch.no_grad():
    for data, targets in tqdm(test_loader):
        data, targets = data.to(device), targets.to(device)
    
        scores = model(data)
        test_loss_total += loss.item()

        preds = torch.softmax(scores, dim=1)[:, 1]
        for m in test_metrics.values():
            m.update(preds, targets)

test_loss_avg = test_loss_total / len(test_loader)
test_stats = {name: m.compute().item() for name, m in test_metrics.items()}

print("\nTEST RESULTS")
print(" ", format_metrics(test_loss_avg, test_stats))

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

all_preds = []
all_labels = []


device = 'cuda'
model.eval()
with torch.no_grad():
    for X, y in val_loader:
        X, y = X.to(device), y.to(device)
        outputs = model(X)
        preds = outputs.argmax(dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(y.cpu().numpy())

precision = precision_score(all_labels, all_preds, average='binary')
recall = recall_score(all_labels, all_preds, average='binary')
f1 = f1_score(all_labels, all_preds, average='binary')

print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")


ViT @ 50 epochs, num_classes = 2, lr = 1e-4, weight_decay = 1e-4:

    [{
        'test_loss': 0.31958553194999695,
        'test_precision': 0.8166184425354004,
        'test_recall': 0.7749434113502502,
        'test_f1score': 0.7925050258636475,
        'test_accuracy': 0.8654947876930237,
        'test_kappa': 0.5859525203704834
    }]

    Notes: took a while to converge, possible still needs more training


ViT @ 150 epochs, num_classes = 2, lr = 1e-4, weight_decay = 1e-4:

    [{'
        'test_loss': 0.38666975498199463,
        'test_precision': 0.882663369178772,
        'test_recall': 0.8467748165130615,
        'test_f1score': 0.8628209829330444,
        'test_accuracy': 0.909375011920929,
        'test_kappa': 0.7259640097618103
    }]  


CCNN @ 50 epochs, num_classes = 2, lr = 1e-4, weight_decay = 1e-4:

    [{
        'test_loss': 0.19597652554512024,
        'test_precision': 0.9394831657409668,
        'test_recall': 0.9228202104568481,
        'test_f1score': 0.9308066368103027,
        'test_accuracy': 0.9529947638511658,
        'test_kappa': 0.8616406917572021
    }]

CCNN @ 12 epochs, num_classes = 2, lr = 1e-4, weight_decay = 1e-4:

    [{
        'test_loss': 0.17744606733322144,
        'test_precision': 0.9170770645141602,
        'test_recall': 0.9295644760131836,
        'test_f1score': 0.9231100082397461,
        'test_accuracy': 0.9459635615348816,
        'test_kappa': 0.8462398052215576
    }]
    